In [62]:
from nltk.tokenize import TweetTokenizer  # módulo para tokenizar los tweets
from nltk.corpus import stopwords  # lista de palabras basura
import unicodedata  # módulo para tratar caracteres unicode
import language_check  # módulo para corregir los tweets
import preprocessor as p  # módulo para limpiar tweets
tknzr = TweetTokenizer()  # selecciona el tokenizador para tweets
tool = language_check.LanguageTool('es')  # trabajador de language_check para correjir
p.set_options(p.OPT.URL)  # hace que preprocessor solo borre las URL dentro de los tweet

In [63]:
# Hacer esto, importante para bajar los data sets de NLTK!

import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [41]:
def strip_accents(s):
    """
    Quita los acentos para que el tokenizador de tweets no parta las palabras con tildes en 2.
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def remove_stopwords(sentence, language):
    """
    Dada una frase, devuelve la frase tokenizada y sin palabras basura
    """
    return [ token for token in tknzr.tokenize(strip_accents(sentence)) if token.lower() not in stopwords.words(language)]

In [42]:
frase = "estamos trists trabajando en análisis de sentimientos"
tokens = remove_stopwords(frase, "spanish")
print(tokens)

['trists', 'trabajando', 'analisis', 'sentimientos']


In [43]:
def correct_word(word):
    """
    Corrije las palabras si es necesario
    """
    matches = tool.check(word)
    return language_check.correct(word, matches)

In [44]:
for palabra in tokens:
    print(correct_word(palabra))

tristes
Trabajando
análisis
Sentimientos


In [47]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("spanish")

In [50]:
def clean_correct_tokenize(text):
    """
    Hace todo el procedimiento de limpiar, correjir, tokenizar, y sacar la raíz de las palabras.
    """
    clean = p.clean(text)
    matches = tool.check(text)
    corrected = language_check.correct(text, matches).lower()
    no_stop_words = remove_stopwords(corrected, 'spanish')
    tokens = [stemmer.stem(correct_word(word).lower()) for word in no_stop_words]
    return tokens

In [51]:
ejemplo = clean_correct_tokenize(frase)
print(ejemplo)

['trist', 'trabaj', 'analisis', 'sentimient']


In [91]:
# Abrir un archivo con texto

with open("don_quijote.txt", "r") as f:
    texto = f.read()
    
frases_preprocesadas = [] 

for sentence in texto[:100000].split('.'):  # Acá tomamos solo 10mil caracteres.
    tokenizado = clean_correct_tokenize(sentence)
    frases_preprocesadas.append(tokenizado)

In [92]:
from gensim.models import Word2Vec

model = Word2Vec(frases_preprocesadas, size=100, window=2, min_count=3, workers=4)

In [93]:
diccionario = {'hola':[1,0,0.5], 'estoy':[0.1,7,9], 'feliz':[-1,-1.1,0]}

for key, value in diccionario.items():
    print(key, value)

hola [1, 0, 0.5]
estoy [0.1, 7, 9]
feliz [-1, -1.1, 0]


In [94]:
len(model.wv.vocab)

759

In [95]:
model.wv.vocab

{'!': <gensim.models.keyedvectors.Vocab at 0x7fca70310c88>,
 '"': <gensim.models.keyedvectors.Vocab at 0x7fca707db048>,
 "'": <gensim.models.keyedvectors.Vocab at 0x7fca7031b198>,
 '(': <gensim.models.keyedvectors.Vocab at 0x7fca707d7320>,
 ',': <gensim.models.keyedvectors.Vocab at 0x7fca70308550>,
 '-': <gensim.models.keyedvectors.Vocab at 0x7fca70310748>,
 ':': <gensim.models.keyedvectors.Vocab at 0x7fca70305cc0>,
 ';': <gensim.models.keyedvectors.Vocab at 0x7fca707d5d68>,
 '?': <gensim.models.keyedvectors.Vocab at 0x7fca702fab38>,
 'abraz': <gensim.models.keyedvectors.Vocab at 0x7fca707d7e80>,
 'abriend': <gensim.models.keyedvectors.Vocab at 0x7fca707db828>,
 'aca': <gensim.models.keyedvectors.Vocab at 0x7fca707db240>,
 'acab': <gensim.models.keyedvectors.Vocab at 0x7fca70310eb8>,
 'acampan': <gensim.models.keyedvectors.Vocab at 0x7fca707d7278>,
 'acas': <gensim.models.keyedvectors.Vocab at 0x7fca70322d30>,
 'acert': <gensim.models.keyedvectors.Vocab at 0x7fca70322e10>,
 'acomet': <

In [98]:
# Semillas:

positivas = ['bien', 'buen']
negativas = ['desgraci', 'mal']

In [102]:
new_positivas = []
new_negativas = []
for pos in positivas:
    similar = model.wv.most_similar(positive=[pos], topn=3)
    for palabra in similar:
        palabra = palabra[0]
        if len(palabra)>1:
            print(palabra)
            new_positivas.append(palabra)
            
for neg in negativas:
    similar = model.wv.most_similar(positive=[neg], topn=3)
    for palabra in similar:
        palabra = palabra[0]
        if len(palabra)>1:
            print(palabra)
            new_negativas.append(palabra)

mas
hab
hac
dese
lueg
caball
mas


In [105]:
positivas += new_positivas
negativas += new_negativas

In [111]:
polaridades = []

for frase in frases_preprocesadas:
    polaridad = 0
    for palabra in frase:
        try:
            model.wv[palabra]
            if palabra in positivas:
                polaridad += 1
            elif palabra in negativas:
                polaridad -= 1
        except:
            pass
    polaridades.append(polaridad)

In [112]:
with open("don_quijote.txt", "r") as f:
    texto = f.read().split('.')[7]
    print(texto)

 Por la cual, por os hacer bien y merced, os damos
licencia y facultad para que vos, o la persona que vuestro poder hubiere, y
no otra alguna, podáis imprimir el dicho libro, intitulado El ingenioso
hidalgo de la Mancha, que desuso se hace mención, en todos estos nuestros
reinos de Castilla, por tiempo y espacio de diez años, que corran y se
cuenten desde el dicho día de la data desta nuestra cédula; so pena que la
persona o personas que, sin tener vuestro poder, lo imprimiere o vendiere,
o hiciere imprimir o vender, por el mesmo caso pierda la impresión que
hiciere, con los moldes y aparejos della; y más, incurra en pena de
cincuenta mil maravedís cada vez que lo contrario hiciere
